# Working with Raster data in Python

## Learning Objectives

 * read spatial raster formats from web services and files
 * write spatial raster formats to disk
 * apply basic operations on raster data
 * plot spatial raster data with matplotlib and rasterio
 * work with categorical rasters


## Raster data

Raster data is stored as a grid of values which are rendered as pixels on a map. Each pixel value represents an area on the Earth’s surface.
Digital photographs are also a raster. A geospatial raster includes spatial information that connects the data to a particular location on Earth. 

![](./img/raster-concept.png)
Source: Colin Williams, NEON

## All good with Seurasaaris trees?

As researchers we often wonder about the weirdest things. 
In this lesson we will wonder about how the trees of [Seurasaari](https://www.hel.fi/helsinki/en/culture/recreation/in-helsinki/seurasaari) (small island in western Helsinki) were doing in end of summer 2021.

And since we do not just wonder but also want to really know, we will see if we can answer this question using some freely available data and open source tools.

Our data: \
-> Earth Observation (ie satellite remote sensing) data: we want to know the situation at a specific time and place. \
-> multispectral remote sensing data: we are interested in spectral information rather than structural. \
-> [Sentinel-2 data](https://sentinel.esa.int/web/sentinel/missions/sentinel-2): Seurasaari is rather small and Sentinel-2 provides up to 10 meter spatial resolution. It also provides data in 12 bands (wavelengths intervals) and the bottom-of-atmosphere (L2A) product is corrected for atmospheric disturbances, which is directly provided by the European Space Agency (ESA).

Our (main) tool:
[Rasterio](https://rasterio.readthedocs.io/en/latest/intro.html) Python package (depends on [GDAL](https://gdal.org/)). 

In [1]:
# Let's import rasterio library (https://rasterio.readthedocs.io/en/latest/)
import rasterio
print(rasterio.__version__)

# and a small toolbox that will help us with the tasks
import seurasaari_toolbox as stb

### Getting the data

There is many tools around to get this data. E.g. [ESAs SciHub](https://scihub.copernicus.eu/dhus/) as graphical web interface. [FORCE](https://force-eo.readthedocs.io/en/latest/) and [sentinelsat](https://sentinelsat.readthedocs.io/en/stable/) are examples of tools that are used from the command line.

Lucky for us, we have a pre-processed geotiff file on Allas ( url: https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif ) that includes 4 band (rgbnir) Sentinel-2 data of 26.09.2021. You can see the preprocessing steps done to this file in `prepare_data.py`.
Let's get it and have a look..

In [ ]:
# Utilize the toolbox to download the data from Allas

s2url = "https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif"
stb.download_data(s2url)


# open the file with rasterio
s2file = "./data/S2B_RGBNIR_20210926_Helsinki.tif"
s2open = rasterio.open(s2file)


> Note: For data on Allas you could also access the data directly without specifically downloading it using vsicurl (virtual system interface for Curl)
like so: `rasterio.open("/vsicurl/https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif")`

### What metadata is available?

In [ ]:
# Check the files metadata
s2open.meta

Affine transformation tells about how the pixels are mapped to a geospatial location with origin, pixel size and rotation of the raster in the geographic coordinate system. 
> Note that bounds, crs and transformation are interrelated and updating one will also affect the others.

The four bands available to us (1:blue, 2:green, 3:red, 4: near infrared) can tell us different things.

![](./img/vegetation_reflectance.png)

Credit: Physicsopenlab, http://physicsopenlab.org/wp-content/uploads/2017/01/veg.gif


### Visualizing the near infrared band

It looks like the near infrared can tell us something about vegetation. Let's visualize it!


In [ ]:
# rasterio provides its own plotting functions
from rasterio.plot import show

# with read we can access the different bands
show(s2open,4)

Great, we can see that our data shows Helsinki, and that we have a range of different reflectances in the near infrared band.

### Common false color image

But there is better ways to look at vegetation. Let's for example have a look at a "common false color image" which utilizes the nir, green and red band to be represented as red, green and blue bands for the image.

In [ ]:
# Read the raster values into numpy arrays
nir = s2open.read(4)
red = s2open.read(3)
green = s2open.read(2)

# check what we got

print(type(nir))
print(nir.dtype)

Uint is chosen for Sentinel-2 data to conserve storage space. To get the reflectance, every pixel value needs to be divided by 10000. Sentinel-2 L2A products no-data value is 0. We can set these 0 values to np.nan instead. These are common processes for satellite remote sensing data. However, the correction factore and nodata values might be different for different sensors. You can usually find these from metadata.

In [ ]:
#for this we need numpy
import numpy as np

# rescaling bands to values between 0 and 1
red = red / 10000
green = green / 10000
nir = nir / 10000

# replacing 0 with np.nan
red[red == 0] = np.nan
green[green == 0] = np.nan
nir[nir == 0] = np.nan
nir.dtype

In [ ]:
#use pyplot imshow for plotting 
#in case of multiband images, it interprets the first band as red, second as green and third as blue 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# we want nir -> red, red -> green and green -> blue
falsecolor_stack = np.dstack((nir, red, green))

# check that it worked
falsecolor_stack.shape

In [ ]:
#and plot it using imshow
plt.imshow(falsecolor_stack)

We divided the values by 10000 to get values between 0 and 1. However, imshow warns us that it had to clip some values for the plot. Let's see what went wrong. 

In [ ]:
# check out min, max and mean of one band
print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))

It seems like we do have some values outside of the [0,1] range. In this case these values are likely some artifacts from preprocessing or atmospheric correction. So we can set all values above 1 to 1.

In [ ]:
#use numpy to set all values > 1 to 1
red[red>1] =1
green[green>1] = 1
nir[nir>1] =1

# and stack again
falsecolor_stack = np.dstack((nir, red, green))

print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))

That looks better.
Let's also look at how our values are distributed with [rasterios histogram plotting function](https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html#rasterio.plot.show_hist) :

In [ ]:
#use rasterio show_hist
from rasterio.plot import show_hist

In [ ]:
# and plot the histogram

show_hist(falsecolor_stack, bins=50, stacked=False, alpha=0.9,histtype="stepfilled")

**Check your understanding:**

* Something seems off about the histogram, can you find out what and fix it?
([Hint](https://rasterio.readthedocs.io/en/latest/topics/plotting.html): check the shape of stack and how rasterio stores it )

* The basic output of the histogram function is not very nice and even wrong in one place, can you do better?
(Hint: show_hist also takes for example matplotlib axes (`ax=`) as optional parameters)

In [ ]:
%load cyu1_solution.txt


It looks like we have a lot of values below 0.5, but using the whole 0-1 range to plot the false color image. This may be the reason it appears so dark.
Let's utilize some image processing (from the toolbox) to disregard very large and very small values and stretch the remaining values across the histogram for visualization purposes.

In [ ]:

stretchnir = stb.stretch(nir)
stretchred = stb.stretch(red)
stretchgreen = stb.stretch(green)

print("before")
print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))
print("after")
print(np.nanmin(stretchred), "-", np.nanmax(stretchred), "mean:", np.nanmean(stretchred))

That looks better, let's plot it!

In [ ]:
#stack and plot histogram stretched image

stretchstack = np.dstack((stretchnir, stretchred, stretchgreen))

plt.imshow(stretchstack)

Much better. Now we can see how green Helsinki is.

### Cropping the image

However, cannot see really well what is going on on Seurasaari.

Since we are not interested in the rest of Helsinki, let's crop out Seurasaari:

In [ ]:
#use osmnx tool for getting a geodataframe with a polygon representing Seurasaari

import osmnx as ox

# Keywords for Seurasaari in such format that they can be found from OSM: https://nominatim.openstreetmap.org/ui/search.html
seurasaari_string = "Seurasaari, Helsinki, Finland"

# Retrieve the geometries of those areas using osmnx and store them in a geodataframe
seurasaari_df = ox.geocode_to_gdf(seurasaari_string)


For the cropping operation we need both the Sentinel-2 data and the Seurasaari polygon in the same Coordinate Reference System (crs):

In [ ]:
#check seurasaaris crs
seurasaari_df.crs

In [ ]:
# reproject polygon to raster crs
seurasaari_df = seurasaari_df.to_crs(crs=s2open.crs)

seurasaari_df.crs

Now we need to get the format of the seurasaari polygon into something usable for rasterio (https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.mask).


In [ ]:
#let's use our toolbox to get the features  
seurasaari_coords = stb.getFeatures(seurasaari_df)
seurasaari_coords

In [ ]:
# use the mask function to crop the Sentinel-2 image
import rasterio.mask

seurasaariS2_array, seurasaariS2_transform = rasterio.mask.mask(dataset=s2open, shapes=seurasaari_coords, crop=True)

# check the shape
seurasaariS2_array.shape


Let's store this cropped version as geotiff.

In [ ]:
# Copy the metadata from the original geotiff
seurasaariS2_meta = s2open.meta.copy()

seurasaariS2_meta.update({"height": seurasaariS2_array.shape[1],
                 "width": seurasaariS2_array.shape[2],
                 "transform": seurasaariS2_transform }
                )

#metadata as keyword arguments (key-value pairs)

seurasaariS2_file = "./data/S2B_RGBNIR_20210926_Seurasaari.tif"

# write the file as geotif
with rasterio.open(seurasaariS2_file, "w", **seurasaariS2_meta) as dest:
        dest.write(seurasaariS2_array)

Now we are done with the Sentinel-2 data of Helsinki. So let's not forget to close it.

In [ ]:
s2open.close()

And we can open the cropped rasterfile again with rasterio and visualize the nir band. 

In [ ]:
# Open the cropped raster file
seurasaariS2 = rasterio.open(seurasaariS2_file)

print(seurasaariS2.count)

# Visualize with colormap defined
show((seurasaariS2, 4), cmap="terrain")

In [ ]:

# use the toolbox to prepare the image (as we did for Helsinki)
ss_falsecolorstack = stb.make_false_color_stack(seurasaariS2)

# and plot it
plt.imshow(ss_falsecolorstack)


Looks like there are a lot of trees on Seurasaari with high reflectances in the near infrared.



### Vegetatiom health indicator

The false color image does give us an impression about the vegetation on Seurasaari, but there is also other ways to look at "vegetation greenness/health".

One often used indicator for vegetation health is the [Normalized Difference Vegetation Index, NDVI](https://gisgeography.com/ndvi-normalized-difference-vegetation-index/) 

![](./img/ndvi.jpg)

Credit: Robert Simmon, NASA, https://www.nasa.gov/topics/earth/features/obscure_data.html

It is calculated as:

$$NDVI = \frac{nir-red}{nir+red}$$

This formula is applied to every single pixel in the raster. So we will need to do some band math / map algebra. And since we can read our data into numpy arrays, let's utilize that and read the nir and red band into arrays:



In [ ]:
#open the Seurasaari geotif 
seurasaariS2 = rasterio.open(seurasaariS2_file)

# make use of the read_band function of the toolbox to read and prepare the arrays
# red is our channel number 3
ss_red = stb.read_band(seurasaariS2,3)
# nir is our channel number 4
ss_nir = stb.read_band(seurasaariS2,4)

# check arrays datatype; range of NDVI is [0,1] -> float
ss_red.dtype

In [ ]:

# since the divisor can be 0, we choose to ignore that error from numpy
np.seterr(divide="ignore", invalid="ignore")


Then we can calculate the NDVI value for every single pixle in our Seurasaari raster:

In [ ]:
#use the formula to calculate NDVI
ndvi = (ss_nir - ss_red) / (ss_nir + ss_red)

# check what is the minimum NDVI value in our raster
np.nanmin(ndvi)
ndvi.dtype

Let's also visualize the NDVI using imshow (currently ndvi is only an array without geoinformation):

In [ ]:
# Plot the NDVI
plt.imshow(ndvi, cmap="viridis")
# Add colorbar to show the index
plt.colorbar()

Looks like the Seurasaari has been green and healthy. Let's store this ndvi array with added geoinformation as geotif:

In [ ]:
#copy metadata from original 
ndvi_meta = seurasaariS2.meta.copy()

#we now only have one band instead of four, and the datatype changed, everything else stays same
ndvi_meta.update({"count" : 1,
                 "dtype" : np.dtype("f8")})

ndvi_file = "./data/S2_NDVI_Seurasaari.tif"

#write the NDVI array to geotif
with rasterio.open(ndvi_file, "w", **ndvi_meta) as dest:
    dest.write_band(1, ndvi.astype(np.dtype("f8")))


**Check your understanding:**

* In above plot we do not have Coordinates at the plot, why? How could we get them?

In [ ]:
%load cyu2_solution.txt

### Collecting more information

In general, it looks like the vegetation on Seurasaari is doing well. But what are those darker areas?

It is possible that not everything is trees on Seurasaari.

Let's check what else there is...

There is many possibilities to do that, we are going to look at the land cover classification [Corine](https://land.copernicus.eu/pan-european/corine-land-cover) , wich SYKE kindly provides for Finland: https://ckan.ymparisto.fi/dataset/%7B0B4B2FAC-ADF1-43A1-A829-70F02BF0C0E5%7D with the categories provided as excel sheet: https://geoportal.ymparisto.fi/meta/julkinen/dokumentit/CorineMaanpeite2018Luokat.xls

For this lesson we will use a cropped versio of the file provided by SYKE, covering Seurasaari.

First, lets visualize Corine data of Seurasaari:

In [ ]:
corine_file = "./data/Clc2018_Seurasaari.tif"


with rasterio.open(corine_file) as corine:
    show(corine,1, cmap = "tab20b")

Up to now we have been working with continuous rasterdata representing reflectance between 0 and 1.
Corine is an example of a categorical raster, with each value representing a land cover class.
From the plot above we can see, that we have multiple different land cover classes on Seurasaari.

Let's see what land cover classes we have on Seurasaari using `zonal_stats` function from [rasterstats](https://pythonhosted.org/rasterstats/).
Note that in this case we could also use numpy to collect values present in the corine array. However, if we had not cropped the original Sentinel-2 file, then we would get the classes of whole Helsinki, in which case using `zonal_stats` makes more sense.

In [ ]:
from rasterstats import zonal_stats

# note that zonal stats takes filenames as well as certain opened file formats
zstats = zonal_stats(seurasaari_df, corine_file, categorical=True)
zstats

For categorical rasters, `zonal_stats` output is a dictionary with categoryvalue (= land cover class) and number of pixels belonging to that class.
Now it would be great to know what these category values mean to find out what is forest. 
Luckily class descriptions are provided as excel file, of which we can pack the interesting (english and detailed) description into a dictionary:

We can add this directly to zonal stats to get the output human reader friendly with classnames instead of class values (https://pythonhosted.org/rasterstats/manual.html#working-with-categorical-rasters):

In [ ]:
#use toolbox function to get a dictionary of land cover class value and its name as category map
cat_dict = stb.get_corine_dict("https://geoportal.ymparisto.fi/meta/julkinen/dokumentit/CorineMaanpeite2018Luokat.xls")
zstats = zonal_stats(seurasaari_df,corine_file, categorical=True, category_map=cat_dict, stats=["count"])


And we can get the pecentage of pixels for each class, by utilizing the 'count' statistics, which counts all valid pixels within the polygon. 

In [ ]:
# use function from toolbox to calculate percentages for each class
stb.get_zonal_stats_percentage(zstats)

Now we know that there are commercial and industrial units on Seurasaari. Note that the Corine file was clipped approximately at the coast, which explains that there is 0.x% of sea and ocean and bare rock, even though if you have ever been to Seurasaari we know it is there.

But we are only interested in trees (forest in Corine case), so lets filter out only forest classes: 'Broad-leaved forest on mineral soil', 'Coniferous forest on mineral soil', 'Coniferous forest on rocky soil', 'Mixed forest on mineral soil', 'Mixed forest on rocky soil'. Let's get the codes of these classes.

In [ ]:
forestcodes = stb.get_forest_codes()

Let's create a mask which shows us where there is forest, and no forest on Seurasaari. 
First we have to read the Corinefile into a numpy array:

In [ ]:
# read the first and only band of the Corine dataset
with rasterio.open(corine_file) as corine:
    corine_array = corine.read(1)
    
corine_array

Now we mask the corinearray everywhere where the values are equal to any of the forest codes:

In [ ]:
forestmask = stb.create_forest_mask(corine_array, forestcodes)

show(forestmask,1)

Now we store the resulting mask as a shapefile using [fiona](https://fiona.readthedocs.io/en/latest/manual.html) (a Python package for reading/writing geographic data files; geopandas would be another option) and [resterio.features](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html):

In [ ]:
# import fiona and rasterio features
import fiona
from rasterio import features


# use features to get the values and connected regions in the mask and store them as polygons with the corresponding raster value as field.
results = ({"properties": {"raster_val": v}, "geometry": s} for i, (s, v) in enumerate(features.shapes(forestmask, transform = corine.transform)))

forestmask_file = "./data/forestmask_corine_seurasaari2.shp"
# use fiona to store the polygons as shapefile with metadata from the corine raster
with fiona.open(forestmask_file, "w", driver="Shapefile", crs=corine.crs, schema={"properties": [("raster_val", "int")], "geometry": "Polygon"}) as dst:
    dst.writerecords(results)

We are only interested in the NDVI values of forests, so we can use the created shapefile to mask out non-forest areas in the NDVI raster. and we can get one value to indicate "how healthy the trees on Seurasaari were in end of September 2021":

In [ ]:

#open the Seurasaari NDVI file and use rasterio mask to mask out areas that are not forest
with rasterio.open(ndvi_file) as ndvi:
    shapefilename = stb.get_reprojected_shapefilename(ndvi.crs, "./data/forestmask_corine_seurasaari2.shp")
    
    # we only want to regard polygons that have the value 1 (forest) that we can get from the raster_val field
    with fiona.open(shapefilename, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile if feature["properties"]["raster_val"] == 1]
    
    #use rasterio.mask again
    ndvi_forest, ndvi_forest_transform = rasterio.mask.mask(ndvi, shapes, crop=True, nodata=np.nan)

print("Mean NDVI value of Seurasaari forests in September 2021:")
print(np.nanmean(ndvi_forest))

Seems like the trees on Seurasaari are green and probably rather healthy :)

## Keypoints

* you can do almost anything to geospatial rasters with rasterio
* single bands can be read into numpy arrays and handled as such for e.g band maths
* be aware when you are working with geocoded data and when you are working with arrays
* you can use rasterio mask to crop a rasterfile with a shapefile
* rasterstats.zonal_statistics is helpful when you want to summarize pixel values from categorical and continuous rasters within polygons

## References

Inspiration for this lesson was drawn from below sources which are also good resources for further reading:

    * Original Raster lesson of this course: https://autogis-site.readthedocs.io/en/latest/lessons/Raster/overview.html (check especially Mosaicing and )
    * Raster data in Python intros: 
        * https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/
        * https://carpentries-incubator.github.io/geospatial-python/05-raster-structure/
        * https://geohackweek.github.io/raster/04-workingwithrasters/
        * https://kodu.ut.ee/~kmoch/geopython2019/L4/raster.html (check out especially the part about reprojecting a raster)
        * https://geoscripting-wur.github.io/PythonRaster/
        * https://pysal.org/scipy2019-intermediate-gds/deterministic/gds2-rasters.html
        * https://geobgu.xyz/py/rasterio.html
        * https://snowex-hackweek.github.io/website/tutorials/geospatial/raster.html
        * https://carpentries-incubator.github.io/geospatial-python/01-intro-raster-data/index.html
    * sample raster values at specific coordinates: https://hackernoon.com/sampling-raster-values-at-specific-coordinates-with-python
    * Virtual Rasters: https://docs.csc.fi/support/tutorials/gis/virtual-rasters/
    * Great collection of Geospatial Python resources: https://github.com/giswqs/python-geospatial